In [ ]:
import numpy as np

def macd_and_strategy_numpy(data, short_window=12, long_window=26, signal_window=9, volume_threshold=500):
    """
    NumPy 기반 MACD 계산과 거래 전략 수행
    :param prices: ndarray, 종가 데이터
    :param volumes: ndarray, 거래량 데이터
    :param short_window: Short EMA 기간 (기본값: 12)
    :param long_window: Long EMA 기간 (기본값: 26)
    :param signal_window: Signal EMA 기간 (기본값: 9)
    :param volume_threshold: 거래량 임계값 (기본값: 500)
    :return: 신호 배열 ('buy', 'sell', None), MACD, Signal, Histogram (ndarray)
    """
    # EMA 계산 함수
    
    prices = data[:,8]
    volumes = data[:, ]
    
    def ema(values, window):
        alpha = 2 / (window + 1)
        ema_values = np.zeros_like(values)
        ema_values[0] = values[0]
        for i in range(1, len(values)):
            ema_values[i] = alpha * values[i] + (1 - alpha) * ema_values[i - 1]
        return ema_values

    # MACD 계산
    ema_short = ema(prices, short_window)
    ema_long = ema(prices, long_window)
    macd = ema_short - ema_long
    signal = ema(macd, signal_window)
    histogram = macd - signal

    # print(ema_short)
    # print(ema_long)
    # print(macd)
    # print(signal)
    # print(histogram)

    # 거래 전략 신호 계산
    buy_signals = (macd > signal) & (np.roll(macd, 1) <= np.roll(signal, 1)) & (volumes > volume_threshold)
    print(buy_signals)
    sell_signals = (macd < signal) & (np.roll(macd, 1) >= np.roll(signal, 1))
    print(sell_signals)

    if buy_signals:
        return 1
    elif sell_signals:
        return 2
    else:
        return 0


# 예제 데이터
prices = np.array([100, 102, 104, 103, 101, 99, 98, 100, 102, 105, 108, 107, 106, 104, 103])
volumes = np.array([500, 520, 480, 600, 620, 450, 430, 500, 550, 580, 600, 620, 610, 590, 580])

# MACD 계산 및 거래 전략 실행
signals, macd, signal, histogram = macd_and_strategy_numpy(prices, volumes)

# 결과 출력
for i, (price, volume, signal_label) in enumerate(zip(prices, volumes, signals)):
    print(f"Index: {i}, Price: {price}, Volume: {volume}, Signal: {signal_label}")


[False False False False False False False False False  True False False
 False False False]
[False False False False False False False False False False False False
 False False False]


TypeError: cannot unpack non-iterable NoneType object

In [3]:
signals

array([None, None, None, None, None, None, None, None, None, 1, None,
       None, None, None, None], dtype=object)

In [4]:
macd

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 3, 3])

In [5]:
signal

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [6]:
histogram

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 3, 3, 3])